[动态图-使用文档-PaddlePaddle深度学习平台](https://www.paddlepaddle.org.cn/documentation/docs/zh/practices/quick_start/dynamic_graph.html)

In [9]:
import paddle
import paddle.nn.functional as F
import numpy as np

print(paddle.__version__)


2.0.0


In [10]:
a = paddle.randn([4, 2])
b = paddle.arange(1, 3, dtype='float32')

print(a)
print(b)

c = a + b
print(c)

print(a.shape)
print(b.shape)

d = paddle.matmul(a, b)
print(d)


Tensor(shape=[4, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 0.05010140,  1.38210380],
        [ 0.93068230,  1.13780129],
        [ 0.09302925, -0.21302903],
        [ 1.60490477,  1.57757306]])
Tensor(shape=[2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [1., 2.])
Tensor(shape=[4, 2], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[1.05010140, 3.38210392],
        [1.93068230, 3.13780117],
        [1.09302926, 1.78697097],
        [2.60490465, 3.57757306]])
[4, 2]
[2]
Tensor(shape=[4], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [ 2.81430888,  3.20628500, -0.33302879,  4.76005077])


In [11]:
a = paddle.to_tensor(np.array([1, 2, 3]))
b = paddle.to_tensor(np.array([4, 5, 6]))

for i in range(10):
    r = paddle.rand([1,])
    if r > 0.5:
        c = paddle.pow(a, i) + b
        print("{} +> {}".format(i, c.numpy()))
    else:
        c = paddle.pow(a, i) - b
        print("{} -> {}".format(i, c.numpy()))


0 -> [-3 -4 -5]
1 -> [-3 -3 -3]
2 +> [ 5  9 15]
3 -> [-3  3 21]
4 +> [ 5 21 87]
5 -> [ -3  27 237]
6 -> [ -3  59 723]
7 -> [  -3  123 2181]
8 -> [  -3  251 6555]
9 -> [   -3   507 19677]


In [12]:
class MyModel(paddle.nn.Layer):
    def __init__(self, input_size, hidden_size):
        super(MyModel, self).__init__()
        self.linear1 = paddle.nn.Linear(input_size, hidden_size)
        self.linear2 = paddle.nn.Linear(hidden_size, hidden_size)
        self.linear3 = paddle.nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        x = self.linear1(inputs)
        x = F.relu(x)

        if paddle.rand([1,]) > 0.5:
            x = self.linear2(x)
            x = F.relu(x)

        x = self.linear3(x)

        return x


In [18]:
total_data, batch_size, input_size, hidden_size = 1000, 64, 128, 256

x_data = np.random.randn(total_data, input_size).astype(np.float32)
y_data = np.random.randn(total_data, 1).astype(np.float32)

model = MyModel(input_size, hidden_size)

loss_fn = paddle.nn.MSELoss(reduction='mean')
optimizer = paddle.optimizer.SGD(learning_rate=0.01,
                                 parameters=model.parameters())

for t in range(200 * (total_data // batch_size)):
    idx = np.random.choice(total_data, batch_size, replace=False)
    x = paddle.to_tensor(x_data[idx,:])
    y = paddle.to_tensor(y_data[idx,:])
    y_pred = model(x)

    loss = loss_fn(input=y_pred, label=y)
    if t % 200 == 0:
        print(t, loss.numpy())

    loss.backward()
    optimizer.step()
    optimizer.clear_grad()


0 [1.425835]
200 [0.7722466]
400 [0.6625432]
600 [0.2778721]
800 [0.12543654]
1000 [0.14481066]
1200 [0.02531685]
1400 [0.01038631]
1600 [0.01894106]
1800 [0.00304903]
2000 [0.00224866]
2200 [0.00243834]
2400 [0.0020945]
2600 [0.0032154]
2800 [0.00281124]


构建更加灵活的网络：共享权重

In [8]:
inputs = paddle.rand((256, 64))

linear = paddle.nn.Linear(64, 8, bias_attr=False)
loss_fn = paddle.nn.MSELoss()
optimizer = paddle.optimizer.Adam(0.01, parameters=linear.parameters())

for i in range(10):
    hidden = linear(inputs)
    # weight from input to hidden is shared with the linear mapping from hidden to output
    outputs = paddle.matmul(hidden, linear.weight, transpose_y=True)
    loss = loss_fn(outputs, inputs)
    loss.backward()
    print("step: {}, loss: {}".format(i, loss.numpy()))
    optimizer.step()
    optimizer.clear_grad()


step: 0, loss: [0.34431243]
step: 1, loss: [0.30196753]
step: 2, loss: [0.27453876]
step: 3, loss: [0.25518516]
step: 4, loss: [0.23617572]
step: 5, loss: [0.21102324]
step: 6, loss: [0.18084018]
step: 7, loss: [0.15068303]
step: 8, loss: [0.12554257]
step: 9, loss: [0.10819472]


In [17]:
list=['wa','zh','liu','test','tes2','eret','33']
for i in range(5):
  np.random.shuffle(list)
print(list)

['liu', 'eret', 'tes2', 'zh', 'test', 'wa', '33']
